## 自定义支持流输出的函数
****
- 当链被使用stream或astream调用的时候
- 如何在链中增加自定义函数

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_deepseek import ChatDeepSeek
import os

llm = ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-V3",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    api_base=os.environ.get("DEEPSEEK_API_BASE"),
)

### 一个简单的链支持流调用

In [6]:
from typing import Iterator, List
from langchain_core.output_parsers import StrOutputParser

# 创建一个聊天提示模板，要求生成5个与给定动物相似的动物名称，以逗号分隔
prompt = ChatPromptTemplate.from_template(
    "请列出5个与以下动物相似的动物名称，用逗号分隔：{animal}。不要包含数字"
)

# 创建一个处理链：提示模板 -> 模型 -> 字符串输出解析器
str_chain = prompt | llm | StrOutputParser()

# 流式输出结果，输入为"熊"
for chunk in str_chain.stream({"animal": "熊"}):
    print(chunk, end="", flush=True)


熊猫, 浣熊, 北极熊, 棕熊, 马来熊

#### 增加自定义函数
****
- 聚合当前流传输的输出
- 在生成下一个逗号的时候组合
- 注意：使用了yield

In [7]:
# 这是一个自定义解析器，将LLM输出的标记迭代器
# 按逗号分隔转换为字符串列表
def split_into_list(input: Iterator[str]) -> Iterator[List[str]]:
    # 保存部分输入直到遇到逗号
    buffer = ""
    for chunk in input:
        # 将当前块添加到缓冲区
        buffer += chunk
        # 当缓冲区中有逗号时
        while "," in buffer:
            # 在逗号处分割缓冲区
            comma_index = buffer.index(",")
            # 输出逗号之前的所有内容
            yield [buffer[:comma_index].strip()]
            # 保存剩余部分用于下一次迭代
            buffer = buffer[comma_index + 1 :]
    # 输出最后一块
    yield [buffer.strip()]


list_chain = str_chain | split_into_list

for chunk in list_chain.stream({"animal": "熊"}):
    print(chunk, flush=True)


['熊猫']
['浣熊']
['树懒']
['袋熊']
['蜜獾']


### yeild与return区别
****
- return 函数立即计算并返回所有结果，而 yield 函数按需计算结果
- return 函数返回一个数据结构（如列表），yield 函数返回一个生成器对象
- yield 函数可以处理潜在的无限序列，而 return 函数必须在有限时间内完成
- 生成器对象是一次性的，遍历完后就被消耗完毕，而 return 返回的数据结构可以重复使用
- yield 特别适合处理大数据集或流式数据，因为它不需要一次性将所有数据加载到内存中

In [11]:
# 使用retun
def get_squares_return(n):
    """返回包含 0 到 n-1 的平方的列表"""
    result = []
    for i in range(n):
        result.append(i * i)
    return result  # 一次性返回所有结果

# 使用 return 函数
squares = get_squares_return(5)
print("使用 return 的结果:", squares)  # 一次性获取所有结果
print("类型:", type(squares))  # 返回类型是列表

# 遍历结果
for num in squares:
    print(num)
#再次遍历
print("-------")
for num in squares:
    print(num)

使用 return 的结果: [0, 1, 4, 9, 16]
类型: <class 'list'>
0
1
4
9
16
-------
0
1
4
9
16


In [9]:
# 使用 yield
def get_squares_yield(n):
    """生成 0 到 n-1 的平方的生成器"""
    for i in range(n):
        yield i * i  # 每次生成一个结果并暂停

# 使用 yield 函数
squares_gen = get_squares_yield(5)
print("使用 yield 的结果:", squares_gen)  # 返回一个生成器对象
print("类型:", type(squares_gen))  # 返回类型是生成器

# 遍历生成器
for num in squares_gen:
    print(num)  # 每次迭代时才计算下一个值

# 再次遍历生成器
print("再次遍历:")
for num in squares_gen:
    print(num)  # 不会输出任何内容，因为生成器已经被消耗完毕

使用 yield 的结果: <generator object get_squares_yield at 0x107f17850>
类型: <class 'generator'>
0
1
4
9
16
再次遍历:
